In [1]:
import os
import base64
import json
import time
from datetime import datetime
import pandas as pd
from openai import OpenAI
from openai.types.chat import ChatCompletionUserMessageParam
from dotenv import load_dotenv

In [2]:
load_dotenv()  
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
PROMPT = """ Please analyze the optic disc photograph I will upload and reply with exactly the following JSON object—nothing else. Use these rules for every field:
• If a value is not discernible, enter "unknown" (or an empty array [] where noted).
• Do not add comments or extra keys.
• Keep number formatting exactly as specified.

{
  "vcdratio (Vertical Cup-Disc Ratio)": "0.0-0.9 in 0.1 increments or 'unknown'",
  "hcdratio (Horizontal Cup-Disc Ratio)": "0.0-0.9 in 0.1 increments or 'unknown'",
  "dischem (Disc Hemorrhage)": "'Yes'/'No'/'unknown'",
  "dhemlocn (Disc Hemorrhage Location)": "['clock hours (e.g., \"7:00\")'] or []",
  "rimnotch (Rim Notch)": "'Yes'/'No'/'unknown'",
  "ntchclkhrs (Rim Notch Location)": "['clock hours (e.g., \"7:30\")'] or []",
  "pericrescent (Peripapillary Atrophy Crescent)": "'Yes'/'No'/'unknown'",
  "rimthin (Rim Thinning)": "'Yes'/'No'/'unknown'",
  "rimthinloc (Rim Thinning Location)": "['clock hour location (e.g., \"7:00\")'] or []",
  "clarity (Image Clarity)": "'Good'/'Fair'/'Poor'/'unknown'",
  "glaucoma (Glaucoma Diagnosis)": "'Yes'/'No'/'unknown'"
}

RULES:
1. Use clock-hour notation (e.g., '7:30') for locations
2. For unclear features, use 'unknown'
3. Empty arrays [] when no locations exist
"""

In [4]:
#Processed keeps tracks of images that are successfully analysed
processed = set() 
results = []       # Stores all results

#Getting all the images
IMAGE_FOLDER = "images"
all_images = {
    f for f in os.listdir(IMAGE_FOLDER) 
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
}

In [5]:
def encode_image(image_name):
    """Convert image to base64 string"""
    img_path = os.path.join(IMAGE_FOLDER, image_name)
    with open(img_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def analyze_image(image_name):
    """Analyze single image and return result"""
    try:
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{
                "role": "user",
                "content": [
                    {"type": "text", "text": PROMPT},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encode_image(image_name)}"
                        }
                    }
                ]
            }],
            response_format={"type": "json_object"}
        )
        
        if not response.choices or not hasattr(response.choices[0], "message"):
            print(f"No choices/message returned for {image_name}")
            return None
        
        generated_content = response.choices[0].message.content

        if not generated_content:
            print(f"No content returned for {image_name}")
            return None
        
        result = json.loads(generated_content)
        result["image_name"] = image_name
        return result
    
    except Exception as e:
        print(f"Error processing {image_name}: {str(e)}")
        return None

In [6]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f'results_{timestamp}'

In [7]:
# Process each image
for img_name in all_images:
    if img_name not in processed:
        result = analyze_image(img_name)
        if result:
            results.append(result)
            processed.add(img_name)
            print(f"Processed: {img_name}")

            #Saves after reach result (In case the script stops unexpectedly)
            pd.DataFrame(results).to_csv(f"{output_filename}.csv", index=False)
        time.sleep(1)  # Avoid rate limits


Processed: 90002-CV822-R.jpg
Processed: 90001-KW254-R.jpg
Processed: 90001-VR219-L.jpg
Processed: 90001-KW254-L.jpg
Processed: 90001-VR219-R.jpg
Processed: 90002-CV822-L.jpg


In [8]:
# Save to Excel
if results:
    df = pd.DataFrame(results)
    # Move 'image_name' to the first column
    cols = df.columns.tolist()
    if "image_name" in cols:
        cols.insert(0, cols.pop(cols.index("image_name")))
        df = df[cols]
    df.to_excel(f"{output_filename}.xlsx", index=False)
    print("Saved results to {output_filename}")

Saved results to {output_filename}


In [ ]:
def merge_two_files(results_file, ground_truth_file = "last_visit_with_img_exist_subcol_v2.csv"):
    """
    Merges the Results file with Ground truth file. 
    Just pass the name of the results_file. 
    For eg:
    results.xlsx
    """

    import pandas as pd
    from datetime import datetime
    import os


    # Load files
    ground_truth_df = pd.read_csv(ground_truth_file)     # First table (CSV)
    results_df = pd.read_excel(results_file)            # ChatGPT results with image_name

    # Extract ran_id, tracking, eye from image_name
    results_df[['ran_id', 'tracking', 'eye']] = results_df['image_name'].str.replace('.jpg', '').str.split('-', expand=True)

    # Convert ran_id to integer to match ground truth
    results_df['ran_id'] = results_df['ran_id'].astype(int)

    # Merge both files on ran_id, tracking, eye
    merged_df = pd.merge(
    ground_truth_df,
    results_df,
    how='left',
    on=['ran_id', 'tracking', 'eye']
    )

    # Optional: Sort to keep left/right eye order consistent
    merged_df = merged_df.sort_values(by=['ran_id', 'eye_no']).reset_index(drop=True)

    # Create timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    # Save merged result with timestamp in filename
    output_filename = f'merged_output_{timestamp}.xlsx'
    merged_df.to_excel(output_filename, index=False)

    print(f"Merged file saved as: {output_filename}")


#Just change the file name and it's done
merge_two_files("results_1.xlsx")